In [1]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [10]:
df = spark.read.parquet("gs://dabi-project-bucket/DWH/DIM_weather/sunshine", inferSchema=True, header=True)
df.createOrReplaceTempView("sunshine")

In [11]:
df_batch_result = spark.sql("SELECT * FROM sunshine")

df_batch_result.toPandas()

,stations_id,Datum,taegliche_Sonnenscheindauer,Tagesmittel_Bedeckungsgrad
0,3631,1994-01-01,0.8,5.3
1,3631,1994-01-02,0.0,7.7
2,3631,1994-01-03,0.1,7.7
3,3631,1994-01-04,0.3,7.7
4,3631,1994-01-05,0.7,7.3
...,...,...,...,...
151114,850,2023-12-27,NaN,NaN
151115,850,2023-12-28,NaN,NaN
151116,850,2023-12-29,NaN,NaN
151117,850,2023-12-30,NaN,NaN


# AVG, MIN, MAX

In [16]:
df_batch_result = spark.sql(
    """
    SELECT 
        YEAR(Datum) as Jahr, 
        MONTH(Datum) as Monat,
        MAX(taegliche_Sonnenscheindauer) as max_Sonnenscheindauer,
        MAX(Tagesmittel_Bedeckungsgrad) as max_Bedeckungsgrad
    FROM 
        sunshine
    GROUP BY 
        Jahr, Monat
    """
)

# In Pandas DataFrame umwandeln
df_batch_result.toPandas()

,Jahr,Monat,max(taegliche_Sonnenscheindauer),max(Tagesmittel_Bedeckungsgrad)
0,1997,11,9.9,8.0
1,2022,10,11.5,8.0
2,1995,12,8.7,8.0
3,2005,5,15.5,8.0
4,1998,7,15.0,8.0
...,...,...,...,...
355,2015,5,14.8,8.0
356,1996,8,13.9,8.0
357,2007,3,11.9,8.0
358,2021,4,14.1,8.0


In [17]:
df_batch_result.write.parquet("gs://dabi-project-bucket/DWH/Analitic_Results/Sunshine/max", mode="overwrite")